In [1]:
import pickle
import numpy as np
import gzip
import torch
import cv2
import math
import random

In [2]:
def get_warp_label(flow1, flow2, label1): #H,W,C  h*w*2
    flow_shape = flow1.shape
    label_shape = label1.shape  
    height, width = flow1.shape[0], flow1.shape[1]

    label2 = torch.zeros(label_shape)   #label2 = np.zeros_like(label1, dtype=label1.dtype)
    flow_t = torch.zeros(flow_shape)    #flow_t = np.zeros_like(flow1, dtype=flow1.dtype)

    #grid = np.indices((height, width)).swapaxes(0, 1).swapaxes(1, 2)
    h_grid = torch.arange(0, height)
    w_grid = torch.arange(0, width)
    h_grid = h_grid.repeat(width, 1).permute(1,0) #.unsqueeze(0)
    w_grid = w_grid.repeat(height,1)              #.unsqueeze(0)
    grid = torch.stack((h_grid,w_grid),0).permute(1,2,0) #float3
    #grid = torch.cat([h_grid, w_grid],0).permute(1,2,0)

    dx = grid[:, :, 0] + flow2[:, :, 1].long()
    dy = grid[:, :, 1] + flow2[:, :, 0].long()
    sx = torch.floor(dx.float()) #float32 #sx = np.floor(dx).astype(int)
    sy = torch.floor(dy.float()) 

    valid = (sx >= 0) & (sx < height - 1) & (sy >= 0) & (sy < width - 1) #H* W 512 x 512 uint8

    # sx_mat = np.dstack((sx, sx + 1, sx, sx + 1)).clip(0, height - 1)
    # sy_mat = np.dstack((sy, sy, sy + 1, sy + 1)).clip(0, width - 1)
    # sxsy_mat = np.abs((1 - np.abs(sx_mat - dx[:, :, np.newaxis])) *
    #                   (1 - np.abs(sy_mat - dy[:, :, np.newaxis])))

    sx_mat = torch.stack((sx, sx + 1, sx, sx + 1),dim=2).clamp(0, height - 1)  #torch.float32
    sy_mat = torch.stack((sy, sy, sy + 1, sy + 1),dim=2).clamp(0, width - 1)
    sxsy_mat = torch.abs((1 - torch.abs(sx_mat - dx.float().unsqueeze(0).permute(1,2,0))) *
                          (1 - torch.abs(sy_mat - dy.float().unsqueeze(0).permute(1,2,0))))

    for i in range(4):
        flow_t = flow_t.long() + sxsy_mat.long()[:, :, i].unsqueeze(0).permute(1,2,0) * flow1.long()[sx_mat.long()[:, :, i], sy_mat.long()[:, :, i], :]

    valid = valid & (torch.norm(flow_t.float()[:, :, [1, 0]] + torch.stack((dx.float(),dy.float()),dim=2) - grid.float(), dim=2, keepdim=True).squeeze(2) < 100)

    flow_t = (flow2.float() - flow_t.float()) / 2.0
    dx = grid.long()[:, :, 0] + flow_t.long()[:, :, 1]
    dy = grid.long()[:, :, 1] + flow_t.long()[:, :, 0]
    valid = valid & (dx >= 0) & (dx < height - 1) & (dy >= 0) & (dy < width - 1)

    label2[valid, :] = label1.float()[dx[valid].long(), dy[valid].long(), :]

    return label2 #HW3

In [3]:
#Original

In [42]:
prev = cv2.imread('0000.png')
cur = cv2.imread('0001.png')
h,w = prev.shape[:2]
flow1 = pickle.loads(gzip.GzipFile('forward_0_1.pkl', 'rb').read()) #'forward_0_5.pkl'
flow2 = pickle.loads(gzip.GzipFile('backward_1_0.pkl', 'rb').read())  #'backward_5_0.pkl'
print("read flow and image")
print(flow1.dtype)
print(prev.shape)

read flow and image
int16
(1080, 1920, 3)


In [43]:
#get_warp_label(forward, backward, 0th)  #  0 -> 1
#get_warp_label(backward, forward, 1th)  #  1 -> 0
tf1 = torch.from_numpy(flow1)
tf2 = torch.from_numpy(flow2)
tcur = torch.from_numpy(cur)
tprev = torch.from_numpy(prev)


In [44]:
w0 = get_warp_label(tf1, tf2, tprev).numpy()  #0->1
w1 = get_warp_label(tf2, tf1, tcur).numpy()  #1->0
cv2.imwrite('from_prev_to_cur.png', w0)
cv2.imwrite('from_cur_to_prev.png', w1)

finish warp


In [45]:
#Resize

In [46]:
fx = 0.3  # fx * 1280(shape[1])
fy = 0.6  # fy * 720(shape[0])
prev_resize = cv2.resize(prev, None, fx=fx, fy=fy, interpolation=cv2.INTER_LINEAR)
cur_resize = cv2.resize(cur, None, fx=fx, fy=fy, interpolation=cv2.INTER_LINEAR)
cv2.imwrite('resize_prev_.png', prev_resize)
cv2.imwrite('resize_cur_.png', cur_resize)
print(prev_resize.shape)

(648, 576, 3)


In [47]:
#target.shape[0] = shape[0] * fy
#target.shape[1] = shape[1] * fx

In [48]:
flow1_resize = cv2.resize(flow1, None, fx=fx, fy=fy, interpolation=cv2.INTER_LINEAR)
flow2_resize = cv2.resize(flow2, None, fx=fx, fy=fy, interpolation=cv2.INTER_LINEAR)

In [49]:
rcur = torch.from_numpy(cur_resize)
rprev = torch.from_numpy(prev_resize)
rf1 = torch.from_numpy(flow1_resize).float()
rf2 = torch.from_numpy(flow2_resize).float()
rf1[:,:,1] *= fy
rf2[:,:,1] *= fy
rf1[:,:,0] *= fx
rf2[:,:,0] *= fx

In [50]:
r0 = get_warp_label(rf1,rf2,rprev).numpy()  #0->1
r1 = get_warp_label(rf2,rf1,rcur).numpy()  #1->0
cv2.imwrite('resize_from_prev_to_cur.png', r0)
cv2.imwrite('resize_from_cur_to_prev.png', r1)
print("finish warp")

finish warp


In [51]:
#Horizontal Flip

In [52]:
h_prev = cv2.flip(prev, 1)
h_cur = cv2.flip(cur, 1)
cv2.imwrite("flip_prev.png", h_prev)
cv2.imwrite("flip_cur.png", h_cur)

True

In [53]:
h_f1 = cv2.flip(flow1, 1)
h_f2 = cv2.flip(flow2, 1)
h_f1[:,:,0] *= -1
h_f2[:,:,0] *= -1

In [54]:
thcur = torch.from_numpy(h_cur)
thprev = torch.from_numpy(h_prev)
thf1 = torch.from_numpy(h_f1)
thf2 = torch.from_numpy(h_f2)

In [55]:
h0 = get_warp_label(thf1, thf2, thprev).numpy()  #0->1
h1 = get_warp_label(thf2, thf1, thcur).numpy()  #1->0
cv2.imwrite('flip_from_prev_to_cur.png', h0)
cv2.imwrite('flip_from_cur_to_prev.png', h1)

True

In [56]:
hw1 = cv2.flip(w1,1)
hw0 = cv2.flip(w0,1)
cv2.imwrite('hw1.png', hw1)
cv2.imwrite('hw0.png', hw0)

True

In [57]:
#Rotation

In [58]:
beta = random.randint(-15,15)
cos = math.cos(math.radians(beta))
sin = math.sin(math.radians(beta))
M = cv2.getRotationMatrix2D((w/2,h/2), beta, 1) #1==resize ratio
new_shape = (w,h)

In [59]:
rot_prev = cv2.warpAffine(prev, M, new_shape)
rot_cur = cv2.warpAffine(cur, M, new_shape)
cv2.imwrite('rotate_prev.png', rot_prev)
cv2.imwrite('rotate_cur.png', rot_cur)

True

In [60]:
#rotate flow
rot_flow1 = cv2.warpAffine(flow1, M, new_shape)
rot_flow2 = cv2.warpAffine(flow2, M, new_shape)
print(rot_flow2.shape)


(1080, 1920, 2)


In [61]:
rotcur = torch.from_numpy(rot_cur)
rotprev = torch.from_numpy(rot_prev)
rotf1 = torch.from_numpy(rot_flow1).float()
rotf2 = torch.from_numpy(rot_flow2).float()

In [62]:
rot = 3
# if rot==1:
#     rotshape = rotf1.shape
#     rotf1 = rotf1.permute(2,0,1).reshape(2,-1)
#     rotf2 = rotf2.permute(2,0,1).reshape(2,-1)
#     rotm = torch.tensor([[cos,sin],[-sin, cos]])
#     rotf1 = rotm.mm(rotf1.float()).reshape(rotshape)
#     rotf2 = rotm.mm(rotf2.float()).reshape(rotshape)
if rot==2:
    rotshape = rotf1.shape
    rotf1 = rotf1.reshape(-1,2)
    rotf2 = rotf2.reshape(-1,2)
    rotm = torch.tensor([[cos,sin,-(1-cos)*w/2-h/2*sin],[-sin, cos,-(1-cos)*h/2+w/2*sin],[0,0,1]])
    rotf1 = rotf1_0.float().mm(rotm).reshape(h,w,3)[:,:,:2]
    rotf2 = rotf2_0.float().mm(rotm).reshape(h,w,3)[:,:,:2]
if rot==3:
    #change flow data
    rotshape = rotf1.shape
    rotf1 = rotf1.reshape(-1,2)
    rotf2 = rotf2.reshape(-1,2)
    rot0 = torch.ones(rotf1.shape[:1]).unsqueeze(1)
    rotf1_0 = torch.cat((rotf1, rot0),dim=1)
    rotf2_0 = torch.cat((rotf2, rot0),dim=1)

    rotm = torch.tensor([[cos,-sin,(1-cos)*w/2+h/2*sin],[sin, cos,(1-cos)*h/2-w/2*sin],[0,0,1]])
    rotf1 = rotf1_0.float().mm(rotm).reshape(h,w,3)[:,:,:2]
    rotf2 = rotf2_0.float().mm(rotm).reshape(h,w,3)[:,:,:2]

print(rotf1.shape)


torch.Size([1080, 1920, 2])


In [63]:
rot0 = get_warp_label(rotf1,rotf2,rotprev).numpy()  #0->1
rot1 = get_warp_label(rotf2,rotf1,rotcur).numpy()  #1->0

cv2.imwrite('rotate_from_prev_to_cur.png', rot0)
cv2.imwrite('rotate_from_cur_to_prev.png', rot1)

True

In [64]:
rotm = np.array(([[cos,-sin,(1-cos)*w/2+h/2*sin],[sin, cos,(1-cos)*h/2-w/2*sin],[0,0,1]]))

In [65]:
rotm

array([[ 9.78147601e-01,  2.07911691e-01, -9.12940097e+01],
       [-2.07911691e-01,  9.78147601e-01,  2.11395519e+02],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [66]:
#THE WHOLE PROCESS

#LOAD IMAGE AND FLOW
prev = cv2.imread('0000.png')
cur = cv2.imread('0001.png')
label = cv2.imread('0000.png')
flow1 = pickle.loads(gzip.GzipFile('forward_0_1.pkl', 'rb').read())  #'forward_0_5.pkl'
flow2 = pickle.loads(gzip.GzipFile('backward_1_0.pkl', 'rb').read())  #'backward_5_0.pkl'
h,w = prev.shape[:2]
#print("INPUT SHAPE:",prev.shape, cur.shape, label.shape, flow1.shape, flow2.shape)
flow1 = flow1.astype(np.float32)
flow2 = flow2.astype(np.float32)


In [67]:
#Augment parameters
resize_rate = 640.0 / min(h,w)
if h < w:
    rh = 640
    rw = int(w * resize_rate)
else:
    rw = 640
    rh = int(h * resize_rate)
resize_shape = (rw, rh)
#print(resize_shape)
crop_size = 512
hflip = random.randint(0,1)  #flip
beta = random.randint(-15,15)  #rotate
cos = math.cos(math.radians(beta))
sin = math.sin(math.radians(beta))
Rotation = cv2.getRotationMatrix2D((rw/2,rh/2), beta, 1) #1==resize ratio
rotm = np.array(([[cos,-sin,(1-cos)*rw/2+rh/2*sin],[sin, cos,(1-cos)*rh/2-rw/2*sin],[0,0,1]]))
rotate_shape = resize_shape

In [68]:
#RESIZE
prev = cv2.resize(prev, resize_shape, interpolation=cv2.INTER_LINEAR)
cur  = cv2.resize(cur, resize_shape, interpolation=cv2.INTER_LINEAR)
flow1 = cv2.resize(flow1, resize_shape, interpolation=cv2.INTER_LINEAR)
flow2 = cv2.resize(flow2, resize_shape, interpolation=cv2.INTER_LINEAR)
flow1 *= resize_rate
flow2 *= resize_rate

In [69]:
#FLIP
if hflip==1:
    prev  = cv2.flip(prev, 1)
    cur   = cv2.flip(cur, 1)
    label = cv2.flip(label, 1)
    flow1 = cv2.flip(flow1, 1)
    flow2 = cv2.flip(flow2, 1)
    flow1[:,:,0] *= -1
    flow2[:,:,0] *= -1


In [225]:
#ROTATE
prev  = cv2.warpAffine(prev, Rotation, rotate_shape)
cur   = cv2.warpAffine(cur, Rotation, rotate_shape)
flow1 = cv2.warpAffine(flow1, Rotation, rotate_shape)
flow2 = cv2.warpAffine(flow2, Rotation, rotate_shape)

rotshape = flow1.shape

flow1 = flow1.reshape(-1,2)
flow2 = flow2.reshape(-1,2)
rot0 = np.ones(flow1.shape[0])[:,np.newaxis]
flow1 =  np.concatenate((flow1, rot0),axis=1)
flow2 =  np.concatenate((flow2, rot0),axis=1)

flow1 = flow1.dot(rotm).reshape(rh,rw,3)[:,:,:2]
flow2 = flow2.dot(rotm).reshape(rh,rw,3)[:,:,:2]


'\nprev = prev.transpose(2,0,1) / 255.0\ncur = cur.transpose(2,0,1) / 255.0\nlabel = (label.transpose(2,0,1)>0).astype(np.int64)\nflow1 = flow1.transpose(2,0,1).astype(np.int64)\nflow2 = flow2.transpose(2,0,1).astype(np.int64)\n'

In [70]:
#CROP
crop_shape = prev.shape[:2] #3,H,W
crop_h = random.randint(0,crop_shape[0]-crop_size)
crop_w = random.randint(0,crop_shape[1]-crop_size)

prev  = prev[crop_h:crop_h+crop_size, crop_w:crop_w+crop_size, :]
cur   = cur[crop_h:crop_h+crop_size, crop_w:crop_w+crop_size, :]
label = label[crop_h:crop_h+crop_size, crop_w:crop_w+crop_size, :]
flow1 = flow1[crop_h:crop_h+crop_size, crop_w:crop_w+crop_size, :]
flow2 = flow2[crop_h:crop_h+crop_size, crop_w:crop_w+crop_size, :]


cv2.imwrite('crop_prev.png', prev)
cv2.imwrite('crop_cur.png', cur)


True

In [71]:
#TO TENSOR
prev = torch.from_numpy(prev).float()#.permute(1,2,0)
cur = torch.from_numpy(cur).float()#.permute(1,2,0)
label = torch.from_numpy(label)#.float()
flow1 = torch.from_numpy(flow1).float()#.permute(1,2,0)
flow2 = torch.from_numpy(flow2).float()#.permute(1,2,0)
print("tensor shape:", prev.shape, cur.shape, label.shape, flow1.shape, flow2.shape)

tensor shape: torch.Size([512, 512, 3]) torch.Size([512, 512, 3]) torch.Size([512, 512, 3]) torch.Size([512, 512, 2]) torch.Size([512, 512, 2])


In [72]:
w0 = get_warp_label(flow1, flow2, prev).numpy()  #0->1
w1 = get_warp_label(flow2, flow1, cur).numpy()  #1->0
cv2.imwrite('crop_from_prev_to_cur.png', w0)
cv2.imwrite('crop_from_cur_to_prev.png', w1)

True